In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf
print(tf.test.gpu_device_name())

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [3]:
import os
import pandas as pd
import numpy as np
import time

import keras.backend as K
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, SGD
from keras.models import Model, Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from keras.applications.xception import Xception, preprocess_input

from keras.models import load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, TensorBoard, CSVLogger

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [4]:
train_dir = 'data/train'
validation_dir = 'data/validation'
test_dir = 'data/test'

In [5]:
for root,dirs,files in os.walk(train_dir):
    print (root, len(files))

data/train 0
data/train/benign 981
data/train/malignant 207


In [6]:
for root,dirs,files in os.walk(validation_dir):
    print (root, len(files))

data/validation 0
data/validation/benign 117
data/validation/malignant 27


In [7]:
for root,dirs,files in os.walk(test_dir):
    print (root, len(files))

data/test 0
data/test/benign 117
data/test/malignant 27


In [8]:
output_classes = 2
batch_size = 32 
img_height, img_width = 256, 256

nb_train_samples = 1188
nb_validation_samples = 144
nb_test_samples = 144

In [9]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [10]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle = True,
    class_mode='binary')

validation_generator = train_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle = True,
    class_mode='binary')

Found 1188 images belonging to 2 classes.
Found 144 images belonging to 2 classes.


In [11]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle = False,
    class_mode='binary')

Found 144 images belonging to 2 classes.


In [12]:
def lr_schedule(epoch):
    if epoch < 15:
        return .01
    elif epoch < 28:
        return .002
    else:
        return .0004

jobs_base_dir = 'jobs'
job_name = 'Xception_job'
model_name = 'Xception'
job_path = "{}/{}".format(jobs_base_dir, job_name)
tensorboard_dir = "{}/{}".format(job_path, "tensorboard")
  
weights_path = "{}/{}".format(job_path, "weights.{epoch:02d}-{val_loss:.2f}.hdf5")
checkpointer = ModelCheckpoint(filepath=weights_path, verbose=1, save_best_only=True)
csv_logger = CSVLogger("{}/{}.log".format(job_path, model_name))
early_stopping = EarlyStopping(monitor='val_loss', verbose=1, patience=25)
tensorboard = TensorBoard(log_dir="{}".format(tensorboard_dir), histogram_freq=0, batch_size=32, write_graph=True,
                          write_grads=False,
                          write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
lr_scheduler = LearningRateScheduler(lr_schedule)

In [13]:
epochs = 50
droput_rate= 0.5
adam_opt = Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-5)

resnet50_weights = 'pretrained-models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [16]:
from keras.applications import ResNet50
from keras.layers import Flatten, Dense, Dropout, BatchNormalization
from keras.regularizers import l2

model = (ResNet50(include_top=False, pooling='avg', weights=resnet50_weights))
x = model.output
# x = Flatten()(x)
x = Dense(1024, activation="relu", kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu", kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation="sigmoid", kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x)
model = Model(input = model.input, output = predictions)

model.compile(loss='binary_crossentropy',
              optimizer= adam_opt,
              metrics = ["accuracy"])

In [ ]:
history = model.fit_generator(train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs = 50,
    validation_data = validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=[lr_scheduler, csv_logger, checkpointer, tensorboard, early_stopping])

Epoch 1/50
37/37 [==============================] - 850s 23s/step - loss: 1.8523 - acc: 0.7230 - val_loss: 4.1222 - val_acc: 0.8125

Epoch 00001: val_loss improved from inf to 4.12216, saving model to jobs/Xception_job/weights.01-4.12.hdf5
Epoch 2/50
37/37 [==============================] - 835s 23s/step - loss: 1.7101 - acc: 0.7601 - val_loss: 3.5040 - val_acc: 0.8125

Epoch 00002: val_loss improved from 4.12216 to 3.50404, saving model to jobs/Xception_job/weights.02-3.50.hdf5
Epoch 3/50
37/37 [==============================] - 851s 23s/step - loss: 1.5920 - acc: 0.7914 - val_loss: 4.1766 - val_acc: 0.6042

Epoch 00003: val_loss did not improve from 3.50404
Epoch 4/50
37/37 [==============================] - 811s 22s/step - loss: 1.4986 - acc: 0.7655 - val_loss: 2.3127 - val_acc: 0.5208

Epoch 00004: val_loss improved from 3.50404 to 2.31273, saving model to jobs/Xception_job/weights.04-2.31.hdf5
Epoch 5/50
37/37 [==============================] - 829s 22s/step - loss: 1.4239 - acc: 

Epoch 37/50
37/37 [==============================] - 826s 22s/step - loss: 0.7954 - acc: 0.8208 - val_loss: 0.7572 - val_acc: 0.8125

Epoch 00037: val_loss improved from 0.75726 to 0.75723, saving model to jobs/Xception_job/weights.37-0.76.hdf5
Epoch 38/50
37/37 [==============================] - 825s 22s/step - loss: 0.7822 - acc: 0.8185 - val_loss: 0.7581 - val_acc: 0.8125

Epoch 00038: val_loss did not improve from 0.75723
Epoch 39/50
37/37 [==============================] - 827s 22s/step - loss: 0.7653 - acc: 0.8093 - val_loss: 0.7572 - val_acc: 0.8125

Epoch 00039: val_loss improved from 0.75723 to 0.75720, saving model to jobs/Xception_job/weights.39-0.76.hdf5
Epoch 40/50
31/37 [========================>.....] - ETA: 2:07 - loss: 0.7576 - acc: 0.8165

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training Acc', 'Test Acc'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training Loss', 'Test Loss'], loc='upper left')
plt.show()

In [ ]:
plt.style.use("ggplot")
plt.figure()
N = epochs
plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history.history["categorical_accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history.history["val_categorical_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")

In [ ]:
score = model.evaluate_generator(validation_generator, steps=50)

print ('Test Score: ', score[0])
print ('Test Accuracy: ',score[1])

In [ ]:
filename = validation_generator.filenames
truth = validation_generator.classes
label = validation_generator.class_indices
indexlabel = dict((value, key) for key, value in label.items())

predicts = model.predict_generator(test_generator, steps=test_generator.samples/test_generator.batch_size, verbose=1)
predict_class = np.argmax(predicts, axis=1)
errors = np.where(predict_class != truth)[0]
print("No of errors = {}/{}".format(len(errors),validation_generator.samples))

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(truth,predict_class)

labels = []
for k,v in indexlabel.items():
    labels.append(v)
    
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion Matrix')

    print(cm)
#     fig = plt.figure()
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.savefig('plots/1.Xception-CM.png', bbox_inches='tight', dpi = 100) 
# fig.savefig('plots/1.Xception-CM.png') 
plt.figure(figsize=(10,10))


plot_confusion_matrix(cm, classes=labels,
                      title='Confusion Matrix')

In [ ]:
#ROC Curve for each class
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
import matplotlib.pyplot as plt
%matplotlib inline  
from scipy import interp
from itertools import cycle

y_pred = predicts
y_pred_probabilities=y_pred

y_pred = np.argmax(y_pred,axis = 1) 
y_actual = validation_generator.classes

classnames=[]
for classname in validation_generator.class_indices:
    classnames.append(classname)

y_actual_binary = label_binarize(y_actual, classes=[0, 1, 2, 3, 4,5,6])
y_pred_binary = y_pred_probabilities#label_binarize(y_pred_probabilities, classes=[0, 1, 2, 3, 4])
n_classes=7
lw=2

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_actual_binary[:, i], y_pred_binary[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_actual_binary.ravel(), y_pred_binary.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure(figsize=(7, 7))


colors = cycle(['red','blue','green','yellow','orange', 'aqua', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(classnames[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic for multi-class')
plt.legend(loc="best")
plt.show()

In [ ]:
# Zoom in view of the upper left corner.
plt.figure(2)
plt.xlim(0, 0.2)
plt.ylim(0.8, 1)

plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=2)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=2)


colors = cycle(['red','blue','green','yellow','orange', 'aqua', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(classnames[i], roc_auc[i]))

    
    
plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic for multi-classs')
plt.legend(loc="lower right")
plt.savefig('plots/1.Xception-ROC.png') 
plt.show()

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report

confusion_mtx = confusion_matrix(y_actual, y_pred) 
print(confusion_mtx)
target_names = classnames
print(classification_report(y_actual, y_pred, target_names=target_names))

In [ ]:
model.save('models/2.ResNet50-Thyroid-Binary-Classification-Model.h5')
model.save_weights('models/2.ResNet50-Thyroid-Binary-Classification-Weights.h5')